In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading data
frame = pd.read_csv('/kaggle/input/clothessizeprediction/final_test.csv')
frame.shape
frame.info()

In [ ]:
#deleting the rows which don't have data
frame = frame.dropna()
frame.info()

In [ ]:
#seeing the data
frame.head()
frame['size'].unique()

In [ ]:
#mapping the size to size_code
frame["size_code"] = frame["size"].map({'S':0,'M':1,'L':2,'XL':3,'XXS':4,'XXL':5,'XXXL':6})
frame.info()

In [ ]:
#correlation matrix to see if any variables are correlated to size
cm = frame.corr()
print(cm)

In [ ]:
#seeing the range of each feature
frame.describe()

In [ ]:
#dropping the size feature as it is string.
frame.drop('size', inplace=True, axis=1)

In [ ]:
frame.info()
frame.head()

In [ ]:
#preparation for prediction. dropping the y label
X = frame
y= frame['size_code']
y_lbl = y.ravel()
print(y_lbl)
X.drop('size_code', inplace=True, axis=1)
print(X.shape)



In [ ]:
#scaling  using min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
 
X_scaled = scaler.fit_transform(X.to_numpy())
X_scaled = pd.DataFrame(X_scaled, columns=['weight', 'age', 'height'])

In [ ]:
#checking mean/value after scaling
X_scaled.shape
X_scaled.describe()

In [ ]:
#test train split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_lbl, test_size=0.2)

In [ ]:
#using decision tree classifier for prediction
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_scaled, y_lbl)
y_lbl_predicted = tree_clf.predict(X_val)

In [ ]:
# Compute accuracy based on test samples
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_val, y_lbl_predicted)
print(acc)

In [ ]:
#Using bagging classifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, n_jobs=-1)
bag_clf.fit(X_scaled, y_lbl)
y_pred = bag_clf.predict(X_val)

In [ ]:
# Compute accuracy based on test samples
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_val, y_pred)
print(acc)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf = RandomForestClassifier()
rnd_clf.fit(X_scaled, y_lbl)
y_pred_rf = rnd_clf.predict(X_val)

In [ ]:
# Compute accuracy based on test samples
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_val, y_pred_rf)
print(acc)